In [7]:
import struct

MASK32 = 0xFFFFFFFF

def rotate_left(v, n):
    return ((v << n) & MASK32) | (v >> (32 - n))

def quarter_round(x, a, b, c, d):
    x[a] = (x[a] + x[b]) & MASK32; x[d] ^= x[a]; x[d] = rotate_left(x[d], 16)
    x[c] = (x[c] + x[d]) & MASK32; x[b] ^= x[c]; x[b] = rotate_left(x[b], 12)
    x[a] = (x[a] + x[b]) & MASK32; x[d] ^= x[a]; x[d] = rotate_left(x[d], 8)
    x[c] = (x[c] + x[d]) & MASK32; x[b] ^= x[c]; x[b] = rotate_left(x[b], 7)

def chacha20_block(key, nonce, counter):
    # 1. Constants (the string "expand 32-byte k")
    state = [
        0x61707865, 0x3320646e, 0x79622d32, 0x6b206574,
        *struct.unpack("<8L", key),
        counter,
        *struct.unpack("<3L", nonce)
    ]
    
    working_state = list(state)
    
    # 2. Perform 20 rounds
    for _ in range(10):
        # Column rounds
        quarter_round(working_state, 0, 4, 8, 12)
        quarter_round(working_state, 1, 5, 9, 13)
        quarter_round(working_state, 2, 6, 10, 14)
        quarter_round(working_state, 3, 7, 11, 15)
        # Diagonal rounds
        quarter_round(working_state, 0, 5, 10, 15)
        quarter_round(working_state, 1, 6, 11, 12)
        quarter_round(working_state, 2, 7, 8, 13)
        quarter_round(working_state, 3, 4, 9, 14)
    
    # 3. Final Addition (Mixing the scrambled state with the original)
    final_state = [(working_state[i] + state[i]) & MASK32 for i in range(16)]
    return struct.pack("<16L", *final_state)

def chacha20_crypt(data, key, nonce):
    result = bytearray()
    for i in range(0, len(data), 64):
        counter = i // 64
        keystream = chacha20_block(key, nonce, counter)
        for j in range(min(64, len(data) - i)):
            result.append(data[i + j] ^ keystream[j])
    return bytes(result)

# --- Test with EXACT lengths ---
key = b"open_sesame_1234_secret_key_32b!" # Exactly 32 bytes
nonce = b"unique_non_1"                    # Exactly 12 bytes
plaintext = b"Then were the king's scribes called at that time in the third month, that is, the month Sivan, on the three and twentieth day thereof; and it was written according to all that Mordecai commanded unto the Jews, and to the lieutenants, and the deputies and rulers of the provinces which are from India unto Ethiopia, an hundred twenty and seven provinces, unto every province according to the writing thereof, and unto every people after their language, and to the Jews according to their writing, and according to their language."

ciphertext = chacha20_crypt(plaintext, key, nonce)
decrypted = chacha20_crypt(ciphertext, key, nonce)

print(f"--- ChaCha20 ---")
print(f"Key Length: {len(key)} bytes\n")
print(f"Ciphertext (hex): {ciphertext.hex()}\n")
print(f"Decrypted: {decrypted.decode()}\n")

--- ChaCha20 ---
Key Length: 32 bytes

Ciphertext (hex): 0f02cf656cd30b0fff415fa7e534230dc00a0d09dc571f283b8e76073ca3902a29432a53013e2d53f3aec3b6529711c6c4e541168e36c30edfe2f193b5658d25a209c0d0d854c37e043e6fe38efe50a2f428ebba5e6226c669317c4901a336db714faa8ec9ec3a89c24180140f0ba5d622b6d62d38933f26dde9f9f94c8d6499d483098efc095e8c361b3c72ffd11abc5448e8987a6307c4e8cadb55d9874d988363bea53b5f1cb20a3ee89a77a9be3875ada62f8975e0b7890957cf12c0e5e4e96284f454bbe98798d356af2cb291b56a4a401e265b43a1579d24c276e191296fc486822cfabbbfc56376c3cdbe671c9f07bcf60277c5b9afbfb5baf648904e743c958c4e9722e91208ead8168129615e69cd5076fbcbe73de2ec8391d189111c916e0ec8df54b306458e52f60f9d43d74e61d1d839ec7f39e414112269234bdb50bcb441d671027541e68125f90e6cc14f671a65904be994431fd7b592712a26159d7311a894eac5d33ab0f97fc3c4effb091f3c6c628dc92c427181bab54ce47327551485921607b4c8ea9b43d1cdab97b1b51b686891c25d995696a1c9dcb5d8296b328c9c4febf85cfb9ca13f497cb8425d4c37639a582b2128f0a55610f89ba1cfeaa673eb1074678a592aff21ad399da3a12cb5d